## Fit GLM-HMM to data from each individual IBL animals.

In [1]:
""" 
IMPORTS
"""
import os
import sys

import autograd.numpy as np

In [2]:
""" 
PARAMETERS
"""
D = 1  # data (observations) dimension
C = 2  # number of output types/categories
N_em_iters = 300  # number of EM iterations
num_bins = 10
USE_CLUSTER = False

# TODO: figure how to use cluster
if USE_CLUSTER:
    z = int(sys.argv[1])
    from glm_hmm_utils import load_cluster_arr, load_session_fold_lookup, \
        load_animal_list, load_data, create_violation_mask, \
        launch_glm_hmm_job
else:
    z = 0
    from glm_hmm_utils import load_cluster_arr, load_session_fold_lookup, \
        load_animal_list, load_data, create_violation_mask, \
        launch_glm_hmm_job

In [3]:
""" 
DIRECTORIES 
"""
global_data_dir = '../../data/ibl/data_for_cluster/'
data_dir = global_data_dir + 'data_by_animal/'
results_dir = '../../results/ibl_individual_fit/'
sys.path.insert(0, '../fit_global_glmhmm/')

data_dir = '/home/ines/repositories/representation_learning_variability/DATA/GLMHMM/data_for_cluster/'
data_dir = '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/GLMHMM/data_for_cluster/'
processed_ibl_data_path = data_dir + "data_by_bin" + str(num_bins) + "global_normalization/"


In [4]:
#  In order to facilitate parallelization of jobs, create a job array that
#  can be used on e.g. a cluster
import numpy as np

prior_sigma = [2]
transition_alpha = [2]
K_vals = [2, 3]
num_folds = 5
N_initializations = 20

if __name__ == '__main__':
    cluster_job_arr = []
    for K in K_vals:
        for i in range(num_folds):
            for j in range(N_initializations):
                for sigma in prior_sigma:
                    for alpha in transition_alpha:
                        cluster_job_arr.append([sigma, alpha, K, i, j])
    np.savez(processed_ibl_data_path + 'cluster_job_arr_individual'
             '.npz',
             cluster_job_arr)
    print(len(cluster_job_arr))


200


In [5]:

# Load external files:
cluster_arr_file = processed_ibl_data_path + 'cluster_job_arr_individual.npz'
# Load cluster array job parameters:
cluster_arr = load_cluster_arr(cluster_arr_file)

for m, model in enumerate(range(len(cluster_arr))):

    [prior_sigma, transition_alpha, K, fold, iter] = cluster_arr[m]
    iter = int(iter)
    fold = int(fold)
    K = int(K)

    animal_list = load_animal_list(processed_ibl_data_path + 'animal_list.npz')
    animal_list = ['DY_013', 'DY_012']
    
    bin_values = [0, 5, 9]

    # Loop through bins and fit a separate global GLM for each bin
    for b, bin in enumerate(bin_values):

        for i, animal in enumerate(animal_list):
            print(animal)

            animal_file = processed_ibl_data_path + animal + '_bin_' + str(bin+1) + '_processed.npz'
            if os.path.exists(animal_file):
                
                bin_fold_lookup_table = load_session_fold_lookup(
                    processed_ibl_data_path + animal + '_' + str(bin+1)+ '_bin_fold_lookup.npz')

                global_fit = False

                inpt, y, session = load_data(animal_file)
                #  append a column of ones to inpt to represent the bias covariate:
                inpt = np.hstack((inpt, np.ones((len(inpt), 1))))
                y = y.astype('int')

                overall_dir = results_dir + animal + '_' + str(bin+1) + '/'

                # Identify violations for exclusion:
                violation_idx = np.where(y == -1)[0]
                nonviolation_idx, mask = create_violation_mask(violation_idx,
                                                            inpt.shape[0])

                init_param_file = processed_ibl_data_path + '/best_global_params' + \
                                '/GLM_HMM_K_' + str(K) + 'bin_' + str(bin+1) + \
                                '/best_params_K_' + \
                                        str(K) + '.npz'

                # create save directory for this initialization/fold combination:
                save_directory = overall_dir + '/GLM_HMM_K_' + str(
                    K) + '/' + 'fold_' + str(fold) + '/' + '/iter_' + str(iter) + '/'
                if not os.path.exists(save_directory):
                    os.makedirs(save_directory)

                launch_glm_hmm_job(inpt, y, session, mask, bin_fold_lookup_table,
                                K, D, C, N_em_iters, transition_alpha, prior_sigma,
                                fold, iter, global_fit, init_param_file,
                                save_directory)


DY_013
Starting inference with K = 2; Fold = 0; Iter = 0
=== fitting GLM-HMM ========


  0%|          | 0/300 [00:00<?, ?it/s]

DY_012
Starting inference with K = 2; Fold = 0; Iter = 0
=== fitting GLM-HMM ========


  0%|          | 0/300 [00:00<?, ?it/s]

DY_013
Starting inference with K = 2; Fold = 0; Iter = 0
=== fitting GLM-HMM ========


...

  0%|          | 0/300 [00:00<?, ?it/s]

DY_013
Starting inference with K = 3; Fold = 4; Iter = 19
=== fitting GLM-HMM ========


  0%|          | 0/300 [00:00<?, ?it/s]

DY_012
Starting inference with K = 3; Fold = 4; Iter = 19
=== fitting GLM-HMM ========


  0%|          | 0/300 [00:00<?, ?it/s]